## OpenPlaning Examples
These are examples that you can find in the FAST '21 paper. It uses the [OpenPlaning](https://github.com/elcf/python-openplaning) package.

### Note
If you launched this notebook via Binder, please note that while this is a fully functioning Jupyter Notebook and you are able to change any variable or function, **none of the changes you make will be saved**. If you want to keep the modifications you made, please download the modified files to your own computer by clicking File > Download as > Notebook (.ipynb).

In [ ]:
from openplaning import PlaningBoat

### Example 1 (Savitsky '76)

In [ ]:
#Vessel particulars
speed = 13.07 #m/s
weight = 827400 #N
beam = 7.315 #m
length = 24.38 #m, vessel LOA
lcg = 10.67 #m, long. center of gravity
vcg = beam/7 #m, vert. center of gravity
r_g = 0.25*length #m, radius of gyration
beta = 15 #deg, deadrise

#Propulsion
epsilon = 0 #deg, thrust angle w.r.t. keel
vT = vcg #m, thrust vertical distance
lT = lcg #m, thrust horizontal distance

#Trim tab particulars
sigma = 1.0 #flap span-hull beam ratio
delta = 5 #deg, flap deflection
Lf = 0.3048 #m, flap chord

#Seaway
H_sig = 1.402 #m, significant wave height

#Create boat object
boat_1 = PlaningBoat(speed, weight, beam, lcg, vcg, r_g, beta, epsilon, vT, lT, length, H_sig, Lf=Lf, sigma=sigma, delta=delta, wetted_lengths_type=3)

#Calculates the equilibrium trim and heave,
#and updates boat.tau and boat.z_wl
boat_1.get_steady_trim()

boat_1.print_description()

### Case Study

The variables are the lcg (constrained to 0.3L -- 0.5L), beta (constrained to 10deg -- 30 deg), and delta (constrained to 0deg -- 15deg). The particulars are similar to those of the RB-M

In [ ]:
# Particulars
speed = 15 #m/s
weight = 162000 #N
beam = 4.5 #m
length = 14 #m
lcg = 0.4*length #m (estimate, 0.3-0.5*length)
vcg = beam/7 #m
r_g = 0.25*length #m
beta = 15 #deg (estimate) (10-30deg)

# Propulsion
epsilon = 5 #deg 
vT = -0.6 #m
lT = 0.7 #m

# Seaway
H_sig = 1 #m, significant wave height (H_sig/b = 0.2-0.7)

# Trim tab
Lf = 0.305 #m, flap chord
sigma = 0.474 #flap span / vessel beam ratio
delta = 1 #deg, flap deflection (estimate) (0-15deg)

#Create boat object
boat_2 = PlaningBoat(speed, weight, beam, lcg, vcg, r_g, beta, epsilon, vT, lT, length, H_sig, Lf=Lf, sigma=sigma, delta=delta)

#Calculates the equilibrium trim and heave for the initial vessel
boat_2.get_steady_trim()

boat_2.print_description()

#### pymoo Problem Definition

For details on setting up the optimization problem, visit https://pymoo.org/getting_started.html

In [ ]:
import numpy as np
from pymoo.model.problem import Problem

from pymoo.algorithms.nsga2 import NSGA2
from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination #https://pymoo.org/interface/termination.html
from pymoo.optimize import minimize
import logging
import time

from pymoo.visualization.scatter import Scatter

In [ ]:
#Normalization values for obj functions
fl = np.array([29500, 2.5])
fu = np.array([33000, 4])
#fl = np.array([0, 0]) #No normalization
#fu = np.array([1, 1])

#Trim angle constraints
tau_l = 2
tau_u = 8

class boatOptProblem(Problem):

    def __init__(self, boat_object):
        
        self.boat = boat_object
        
        super().__init__(n_var=3,
                         n_obj=2,
                         n_constr=4,
                         xl=np.array([4.2,10,0]),
                         xu=np.array([7,30,15]),
                         elementwise_evaluation=True)
        
    def _evaluate(self, x, out, *args, **kwards):
        
        #Optimization variables
        [self.boat.lcg, self.boat.beta, self.boat.delta] = x
        
        #Attempt to get steady trim, and fail inequality constraint if not successful
        g4 = 0
        try:
            self.boat.get_steady_trim()
        except RuntimeError:
            g4 = 1
            
        #Run the function needed for seakeeping
        self.boat.get_seaway_behavior()
        
        #Objective functions and normalization
        f1 = ((self.boat.net_force[0] + self.boat.R_AW) - fl[0])/(fu[0]-fl[0])
        f2 = (self.boat.avg_impact_acc[1] - fl[1])/(fu[1]-fl[1])
        
        #Inequality constraints (g(x)<=0)
        g1 = self.boat.tau - tau_u
        g2 = tau_l - self.boat.tau
        g3 = self.boat.L_K - self.boat.length*.9
        
        out['F'] = [f1, f2]
        out['G'] = [g1, g2, g3, g4]
    
#Create opt problem object
boat_problem = boatOptProblem(boat_2)

##### Algorithm selection and run optimization

In [ ]:
algorithm = NSGA2(pop_size=200)
termination = MultiObjectiveSpaceToleranceTermination(tol=0.0025,
                                                      n_last=20,
                                                      nth_gen=5,
                                                      n_max_gen=500,
                                                      n_max_evals=None)

#Run optimization
logging.captureWarnings(True) #Silence warnings
start_time = time.time()
result = minimize(boat_problem,
                algorithm,
                termination,
                seed=1,
                verbose=True)
elapsed_time = time.time() - start_time
logging.captureWarnings(False)
print(f"The elapsed wall time was {elapsed_time:.0f} s")

#### Visualization of results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#Creating a dataframe for convenient data manipulation
df = pd.DataFrame(result.X, columns = ['LCG (m)', 'Deadrise (deg)', 'Trim Tab Angle (deg)'])
df[['Tot. Seaway Drag/Weight','Average Bow Acc. (g)']] = pd.DataFrame((result.F*(fu - fl) + fl)/[weight, 1])
df[['tau - '+str(tau_u), str(tau_l)+' - tau','L_K - 0.9*length', 'Steady Trim Fail']] = pd.DataFrame(result.G)
df['Calm Water Trim (deg)'] =  df['tau - '+str(tau_u)] + tau_u
df

In [ ]:
#Plot font size
FONT_SIZE = 11.5
plt.rc('font', size=FONT_SIZE) # controls default text sizes
plt.rc('axes', titlesize=FONT_SIZE) # fontsize of the axes title
plt.rc('axes', labelsize=FONT_SIZE) # fontsize of the x and y labels
plt.rc('xtick', labelsize=FONT_SIZE) # fontsize of the tick labels
plt.rc('ytick', labelsize=FONT_SIZE) # fontsize of the tick labels
plt.rc('legend', fontsize=FONT_SIZE) # legend fontsize
plt.rc('figure', titlesize=FONT_SIZE) # fontsize of the figure title

In [ ]:
fig_1, axs_1 = plt.subplots(figsize=[5,5])
df.plot.scatter('Tot. Seaway Drag/Weight', 'Average Bow Acc. (g)', sharex=False, ax=axs_1);
axs_1.grid(True)
# fig_1.savefig('pareto.png')

In [ ]:
fig_2, axs_2 = plt.subplots(3, 1, figsize=[5,10])
df.plot.scatter('Tot. Seaway Drag/Weight', 'Average Bow Acc. (g)', c='LCG (m)', colormap='viridis', sharex=True, ax=axs_2[0]);
df.plot.scatter('Tot. Seaway Drag/Weight', 'Average Bow Acc. (g)', c='Deadrise (deg)', colormap='plasma', ax=axs_2[1]);
df.plot.scatter('Tot. Seaway Drag/Weight', 'Average Bow Acc. (g)', c='Trim Tab Angle (deg)', colormap='cividis', ax=axs_2[2]);
axs_2[0].set_ylabel(None);
axs_2[2].set_ylabel(None);
fig_2.tight_layout()
# fig_2.savefig('pareto_vars.png')

In [ ]:
fig_3, axs_3 = plt.subplots(3, 1, figsize=[5,12])
color_values = 'Tot. Seaway Drag/Weight'
df.plot.scatter('LCG (m)', 'Deadrise (deg)', c=color_values, colormap='magma', sharex=False, ax=axs_3[0], colorbar=False);
df.plot.scatter('LCG (m)', 'Trim Tab Angle (deg)', c=color_values, colormap='magma', ax=axs_3[1], colorbar=False);
df.plot.scatter('Deadrise (deg)', 'Trim Tab Angle (deg)', c=color_values, colormap='magma', ax=axs_3[2], colorbar=False);
im = plt.gca().get_children()[0]
cbar = fig_3.colorbar(im, ax=axs_3.ravel().tolist(), aspect=60)
cbar.set_label(color_values)
# fig_3.savefig('vars.png')